In [2]:
import math
import random
import os

import igl
import numpy as np
import meshplot as mp
import scipy as sp
import ipywidgets as iw

from PIL import Image

In [3]:
from scipy.optimize import minimize

# Extract Feature Lines with threshold for crease detection

In [4]:
def feature_edges(v,f,threshold):
    edge_faces = []

    dihedral_threshold = threshold/180 * np.pi
    f_normals = igl.per_face_normals(v,f,np.ndarray([0,0]))
    edge_flaps = igl.edge_flaps(f)
    edges = edge_flaps[0]
    
    threshold_edges = []
    for i in range(edges.shape[0]):
        # find normals of faces connecting edge
        edge_face_norms = f_normals[edge_flaps[2][i]]

        # find dihedral angle between edge faces
        f1n = edge_face_norms[0] / np.linalg.norm(edge_face_norms[0])
        f2n = edge_face_norms[1] / np.linalg.norm(edge_face_norms[1])
        dihedral_angle = np.arccos(np.clip(np.dot(f1n, f2n), -1.0, 1.0))

        # add angles greater than threshold to list
        if dihedral_angle > dihedral_threshold or (-1 in edge_flaps[2][i]):
            threshold_edges.append(edges[i])
            
            # add edge faces to list
            if edge_flaps[2][i][0] != -1:
                edge_faces.append(edge_flaps[2][i][0])
            if edge_flaps[2][i][1] != -1:
                edge_faces.append(edge_flaps[2][i][1])

    threshold_edges = np.array(threshold_edges)
    return threshold_edges, edge_faces



v, f = igl.read_triangle_mesh("data/camel_head.off")
feature_edge_list, edge_faces = feature_edges(v,f,10)

col = np.ones_like(f)
col[edge_faces, 1:] = 0

p = mp.plot(v, f, c=col, shading={"wireframe": 0})
p.add_lines(v[feature_edge_list][:,0], v[feature_edge_list][:,1], shading={"line_color": "yellow"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

1

# Parameterize mesh into UV plane (Harmonic)

In [5]:
def circle_to_square(x_circ,y_circ):
    x = 0.5 * np.sqrt( 2 + x_circ**2 - y_circ**2 + 2*x_circ*np.sqrt(2) ) - 0.5 * np.sqrt( 2 + x_circ**2 - y_circ**2 - 2*x_circ*np.sqrt(2) )
    y = 0.5 * np.sqrt( 2 - x_circ**2 + y_circ**2 + 2*y_circ*np.sqrt(2) ) - 0.5 * np.sqrt( 2 - x_circ**2 + y_circ**2 - 2*y_circ*np.sqrt(2) )
    x = round(max(min(1,x),-1),5)
    y = round(max(min(1,y),-1),5)
    return [x,y]

def parametrize(v,f):
    ## Find the open boundary
    bnd = igl.boundary_loop(f)
    
    ## Map the boundary to a circle, preserving edge proportions
    bnd_uv = igl.map_vertices_to_circle(v, bnd)
    
    # remap circle boundary onto square
    bnd_uv = np.array([circle_to_square(x_circ,y_circ) for [x_circ,y_circ] in bnd_uv])

    # force points into corners
    TL = np.array([-1,1])
    TR = np.array([1,1])
    BL = np.array([-1,-1])
    BR = np.array([1,-1])
    bndTL = 0
    bndTR = 0
    bndBL = 0
    bndBR = 0
    for coordi, coord in enumerate(bnd_uv):
        if np.linalg.norm(coord-TL) < np.linalg.norm(bnd_uv[bndTL]-TL):
            bndTL = coordi
        if np.linalg.norm(coord-TR) < np.linalg.norm(bnd_uv[bndTR]-TR):
            bndTR = coordi
        if np.linalg.norm(coord-BL) < np.linalg.norm(bnd_uv[bndBL]-BL):
            bndBL = coordi
        if np.linalg.norm(coord-BR) < np.linalg.norm(bnd_uv[bndBR]-BR):
            bndBR = coordi

    bnd_uv[bndTL] = TL
    bnd_uv[bndTR] = TR
    bnd_uv[bndBL] = BL
    bnd_uv[bndBR] = BR
    
    ## Harmonic parametrization for the internal vertices
    return igl.harmonic(v, f, bnd, bnd_uv, 1)

v, f = igl.read_triangle_mesh("data/camel_head.off")
uv = parametrize(v,f)
feature_edge_list, _ = feature_edges(v,f,20)


p = mp.plot(v, f, c=np.ones_like(f), shading={"wireframe": False})
p.add_lines(v[feature_edge_list][:,0], v[feature_edge_list][:,1], shading={"line_color": "yellow"})
@iw.interact(mode=['2D','3D'])
def switch(mode):
    if mode == "3D":
        mp.plot(v, f, c=uv[:,0],shading={"wireframe": 0}, plot=p)
        p.add_lines(v[feature_edge_list][:,0], v[feature_edge_list][:,1], shading={"line_color": "yellow"})
        
    if mode == "2D":
        mp.plot(uv, f, c=uv[:,0], shading={"wireframe": 1, "line_width":0.2}, plot=p)
        p.add_lines(uv[feature_edge_list][:,0], uv[feature_edge_list][:,1], shading={"line_color": "yellow"})
        p.add_points(uv, shading={"point_size": .04})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

interactive(children=(Dropdown(description='mode', options=('2D', '3D'), value='2D'), Output()), _dom_classes=…

# Area Distortion

In [6]:
def normalizeZeroToOne(arr):
    if(min(arr) < 0):
        arr -= min(arr)
    return (1/max(arr)) * arr
    
def areaDistortion(v,uv,f):
    a3d = igl.doublearea(v,f)
    a2d = igl.doublearea(uv,f)

    VF, NI = igl.vertex_triangle_adjacency(f,v.shape[0])
    face_area_distortion = a3d/a2d

    vert_area_distortion = np.zeros_like(v[:,0])
    for i in range(v.shape[0]):
        face0 = NI[i]
        facen = NI[i+1]
        faces = VF[face0:facen]
        faceAreas = a3d[faces]
        faceAreas = (1/np.sum(faceAreas)) * faceAreas
        #print(i, faces,faceAreas)

        vertAD = 0
        for facei, face in enumerate(faces):
            vertAD += face_area_distortion[face] * faceAreas[facei]
        vert_area_distortion[i] = vertAD

    return normalizeZeroToOne(vert_area_distortion)


v, f = igl.read_triangle_mesh("data/camel_head.off")
uv = parametrize(v,f)
feature_edge_list, _ = feature_edges(v,f,10)
area_distortion = areaDistortion(v,uv,f)

a = mp.plot(uv, f, c=area_distortion, shading={"wireframe": 0, "line_width":0.2})
a.add_lines(uv[feature_edge_list][:,0], uv[feature_edge_list][:,1], shading={"line_color": "black"})
a.add_points(uv, shading={"point_size": .03, "point_color": "black"})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

2

# Minimize area distortion

In [7]:
def optimize_uv(v, uv, f):
    a3d = igl.doublearea(v, f) / 2.0
    def objective(uv_flat):
        uv = uv_flat.reshape(-1, 2)
        a2d = igl.doublearea(uv, f) / 2.0 
        return np.sum((a3d - a2d)**2) 

    result = minimize(objective, uv.ravel(), method='L-BFGS-B', options={'disp': True})
    
    return result.x.reshape(-1, 2)
optimized_uv = optimize_uv(v, uv, f)



In [8]:
area_distortion = areaDistortion(v, optimized_uv, f)
a = mp.plot(optimized_uv, f, c=area_distortion, shading={"wireframe": 0, "line_width":0.2})
a.add_lines(optimized_uv[feature_edge_list][:,0], optimized_uv[feature_edge_list][:,1], shading={"line_color": "black"})
a.add_points(optimized_uv, shading={"point_size": .03, "point_color": "black"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0074865…

2

# Triangulate UV plane nicely

Constrained edges and minimized distortion

In [9]:
def featureGraph(v,f):
    feature_edge_list, _ = feature_edges(v,f,20)
    bnd = igl.boundary_loop(f)
    feature_graph = igl.map_vertices_to_circle(v, bnd)
    feature_graph = np.array([circle_to_square(x_circ,y_circ) for [x_circ,y_circ] in feature_graph])
    for coord in uv[feature_edge_list]:
        feature_graph = np.append(feature_graph, coord, axis=0)
    return feature_graph

remeshUV=optimized_uv
feature_graph = featureGraph(v,f)

remeshUV = np.append(remeshUV, feature_graph, axis=0)
remeshF = sp.spatial.Delaunay(remeshUV).simplices

p=mp.plot(remeshUV, remeshF, shading={"wireframe": 1, "line_width":0.9})
p.add_points(remeshUV, shading={"point_size": .06, "point_color": "black"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

# Optional: Increase Samplings for High Curvature

In [10]:
def curveMaps(v,f):
    gaussCurv = igl.gaussian_curvature(v,f)
    _, _, maxCurv, minCurv = igl.principal_curvature(v, f)
    totalCurv = maxCurv * maxCurv + minCurv * minCurv

    gaussCurv = normalizeZeroToOne(gaussCurv * gaussCurv)
    maxCurv = normalizeZeroToOne(maxCurv)
    minCurv = normalizeZeroToOne(minCurv)
    totalCurv = normalizeZeroToOne(totalCurv)

    return gaussCurv, maxCurv, minCurv, totalCurv


v, f = igl.read_triangle_mesh("data/camel_head.off")
uv = parametrize(v,f)
feature_edge_list, _ = feature_edges(v,f,10)
gaussCurv, maxCurv, minCurv, totalCurv = curveMaps(v,f)

a = mp.plot(uv, f, c=gaussCurv, shading={"wireframe": 0, "line_width":0.2})
a.add_lines(uv[feature_edge_list][:,0], uv[feature_edge_list][:,1], shading={"line_color": "black"})
a.add_points(uv, shading={"point_size": .03, "point_color": "black"})
@iw.interact(mode=['Gaussian','Max','Min','Total'])
def switch(mode):
    if mode == "Gaussian":
        mp.plot(uv, f, c=gaussCurv, shading={"wireframe": 0, "line_width":0.2}, plot=a)
        a.add_lines(uv[feature_edge_list][:,0], uv[feature_edge_list][:,1], shading={"line_color": "black"})
        a.add_points(uv, shading={"point_size": .03, "point_color": "black"})
        
    if mode == "Max":
        mp.plot(uv, f, c=maxCurv, shading={"wireframe": 0, "line_width":0.2}, plot=a)
        a.add_lines(uv[feature_edge_list][:,0], uv[feature_edge_list][:,1], shading={"line_color": "black"})
        a.add_points(uv, shading={"point_size": .03, "point_color": "black"})
        
    if mode == "Min":
        mp.plot(uv, f, c=minCurv, shading={"wireframe": 0, "line_width":0.2}, plot=a)
        a.add_lines(uv[feature_edge_list][:,0], uv[feature_edge_list][:,1], shading={"line_color": "black"})
        a.add_points(uv, shading={"point_size": .03, "point_color": "black"})
        
    if mode == "Total":
        mp.plot(uv, f, c=totalCurv, shading={"wireframe": 0, "line_width":0.2}, plot=a)
        a.add_lines(uv[feature_edge_list][:,0], uv[feature_edge_list][:,1], shading={"line_color": "black"})
        a.add_points(uv, shading={"point_size": .03, "point_color": "black"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(Dropdown(description='mode', options=('Gaussian', 'Max', 'Min', 'Total'), value='Gaussia…

In [14]:
def subdivide_high_curvature_regions(v, f, curvature, threshold=0.5, iterations=1):
    
    # Identify vertices with high curvature
    high_curvature_vertices = np.where(curvature > threshold)[0]
    mask = np.zeros(len(v), dtype=bool)
    mask[high_curvature_vertices] = True

    # Subdivide the mesh, focusing on high curvature areas
    for _ in range(iterations):
        v_new, f_new = v.copy(), f.copy()
        for fi in range(len(f)):
            if mask[f[fi, 0]] or mask[f[fi, 1]] or mask[f[fi, 2]]:
                # Apply a simple subdivision rule (splitting faces)
                v_mid = np.mean(v[f[fi]], axis=0)
                v_new = np.append(v_new, [v_mid], axis=0)
                mid_idx = len(v_new) - 1
                # Create new faces
                f_new = np.append(f_new, [[f[fi, 0], f[fi, 1], mid_idx]], axis=0)
                f_new = np.append(f_new, [[f[fi, 1], f[fi, 2], mid_idx]], axis=0)
                f_new = np.append(f_new, [[f[fi, 2], f[fi, 0], mid_idx]], axis=0)
                # Remove the original face
                f_new = np.delete(f_new, fi, axis=0)

        v, f = v_new, f_new

    return v, f

v, f = igl.read_triangle_mesh("data/camel_head.off")
uv = parametrize(v, f)
feature_edge_list, _ = feature_edges(v, f, 10)
gaussCurv, maxCurv, minCurv, totalCurv = curveMaps(v, f)

# Example: Subdividing based on Gaussian curvature
v_subdivided, f_subdivided = subdivide_high_curvature_regions(v, f, gaussCurv, threshold=0.5, iterations=1)

mp.plot(v_subdivided, f_subdivided, c='blue', shading={"wireframe": True})

Invalid color array given! Supported are numpy arrays. <class 'str'>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…